# New Section

In [1]:

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import datetime

import joblib
import pickle
import tempfile


import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
''''
calander = pd.read_csv('s3://{}/{}'.format(bucket, 'calendar.csv'))
sales_train_evaluation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_evaluation.csv'))
sales_train_validation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_validation.csv'))
sample_submission = pd.read_csv('s3://{}/{}'.format(bucket, 'sample_submission.csv'))
sell_prices = pd.read_csv('s3://{}/{}'.format(bucket, 'sell_prices.csv'))
'''


"'\ncalander = pd.read_csv('s3://{}/{}'.format(bucket, 'calendar.csv'))\nsales_train_evaluation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_evaluation.csv'))\nsales_train_validation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_validation.csv'))\nsample_submission = pd.read_csv('s3://{}/{}'.format(bucket, 'sample_submission.csv'))\nsell_prices = pd.read_csv('s3://{}/{}'.format(bucket, 'sell_prices.csv'))\n"

**Load the data**

In [59]:
sales_train_evaluation = pd.read_csv('s3://{}/{}'.format(bucket, 'sales_train_evaluation.csv'))
calander = pd.read_csv('s3://{}/{}'.format(bucket, 'calendar.csv'))
sell_prices = pd.read_csv('s3://{}/{}'.format(bucket, 'sell_prices.csv'))

In [60]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [61]:
sales_train_evaluation = reduce_mem_usage(sales_train_evaluation)

Mem. usage decreased to 96.13 Mb (78.8% reduction)


In [62]:
calandar = reduce_mem_usage(calander)

Mem. usage decreased to  0.12 Mb (41.9% reduction)


In [63]:
sell_prices = reduce_mem_usage(sell_prices)

Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [64]:
sales_train_evaluation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1,0,1,4,0,0,5,0,0,0,0,0,0,2,1,2,1,0,0,0,1,1,1,0,0,1,1,1,1

In [65]:
calandar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [66]:
sell_prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [67]:
nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    calandar[feature].fillna('unknown', inplace = True)

In [68]:
calandar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,unknown,unknown,unknown,unknown,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,unknown,unknown,unknown,unknown,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,unknown,unknown,unknown,unknown,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,unknown,unknown,unknown,unknown,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,unknown,unknown,unknown,unknown,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,unknown,unknown,unknown,unknown,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,unknown,unknown,unknown,unknown,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,unknown,unknown,unknown,unknown,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,unknown,unknown,unknown,unknown,0,0,0


In [69]:
cat = ['item_id','dept_id','cat_id','store_id','state_id']
numcols = [f"d_{day}" for day in range(1,1941)]

In [70]:
sales_train_evaluation = pd.melt(sales_train_evaluation, id_vars=cat, value_vars = numcols, var_name='d',value_name='demand')

In [71]:
sales_train_evaluation

,item_id,dept_id,cat_id,store_id,state_id,d,demand
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...
59150595,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1940,1
59150596,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1940,1
59150597,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1940,0
59150598,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1940,1


In [72]:
sales_train_evaluation = sales_train_evaluation.merge(calander,on='d')

In [73]:
sales_train_evaluation = sales_train_evaluation.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'])

In [74]:
sales_train_evaluation['d'] =  sales_train_evaluation['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [75]:
calander['d'] =  calander['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [76]:
sales_train_evaluation = sales_train_evaluation.drop(columns=['weekday'])

In [77]:
cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in cat:
    encoder = LabelEncoder()
    sales_train_evaluation[feature] = encoder.fit_transform(sales_train_evaluation[feature])

In [78]:
sales_train_evaluation

,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,1444,3,1,0,0,1,12,2011-01-29,11101,1,1,2011,30,4,4,2,0,0,0,0.459961
1,1444,3,1,0,0,2,15,2011-01-30,11101,2,1,2011,30,4,4,2,0,0,0,0.459961
2,1444,3,1,0,0,3,0,2011-01-31,11101,3,1,2011,30,4,4,2,0,0,0,0.459961
3,1444,3,1,0,0,4,0,2011-02-01,11101,4,2,2011,30,4,4,2,1,1,0,0.459961
4,1444,3,1,0,0,5,0,2011-02-02,11101,5,2,2011,30,4,4,2,1,0,1,0.459961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46851182,1432,2,0,9,2,1940,1,2016-05-21,11617,1,5,2016,30,4,4,2,0,0,0,2.980469
46851183,1433,2,0,9,2,1940,1,2016-05-21,11617,1,5,2016,30,4,4,2,0,0,0,2.480469
46851184,1434,2,0,9,2,1940,0,2016-05-21,11617,1,5,2016,30,4,4,2,0,0,0,3.980469
46851185,1435,2,0,9,2,1940,1,2016-05-21,11617,1,5,2016,30,4,4,2,0,0,0,1.280273


In [80]:

data_dir = ''

sales_train_evaluation.to_csv(os.path.join(data_dir, 'sales_train_evaluation_processed.csv'), index=False)

sales_train_evaluation_location = sagemaker_session.upload_data(os.path.join(data_dir, 'sales_train_evaluation_processed.csv'))



In [2]:
sales_train_evaluation_lag_rolling = pd.read_csv('s3://{}/{}'.format(bucket, 'data/sales_train_evaluation_processed.csv'))

In [3]:
sales_train_evaluation_lag_rolling['date'] = pd.to_datetime(sales_train_evaluation_lag_rolling['date'])
sales_train_evaluation_lag_rolling['date_day'] = sales_train_evaluation_lag_rolling['date'].dt.day.astype(np.int8)
sales_train_evaluation_lag_rolling['date_week'] = sales_train_evaluation_lag_rolling['date'].dt.week.astype(np.int8)
sales_train_evaluation_lag_rolling['date_month'] = sales_train_evaluation_lag_rolling['date'].dt.month.astype(np.int8)
sales_train_evaluation_lag_rolling['date_year'] = sales_train_evaluation_lag_rolling['date'].dt.year
sales_train_evaluation_lag_rolling['date_year'] = (sales_train_evaluation_lag_rolling['date_year'] - sales_train_evaluation_lag_rolling['date_year'].min()).astype(np.int8)


/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  app.launch_new_instance()


NameError: name 'ceil' is not defined

In [4]:
from math import ceil
sales_train_evaluation_lag_rolling['date_week_m'] = sales_train_evaluation_lag_rolling['date_day'].apply(lambda x: ceil(x/7)).astype(np.int8)

In [5]:
from tqdm import tqdm

#lag features
lags = [28, 35, 42, 49, 56]
for lag in tqdm(lags):
    sales_train_evaluation_lag_rolling['lag_' + str(lag)] = sales_train_evaluation_lag_rolling.groupby('item_id')['demand'].shift(lag).astype(np.float16)

100%|██████████| 4/4 [00:05<00:00,  1.50s/it]


In [7]:
#rolling median features
sales_train_evaluation_lag_rolling['lag_28'] = sales_train_evaluation_lag_rolling.groupby('item_id')['demand'].transform(lambda x: x.shift(28)).astype(np.float16) 
for i in [7,14,30,60,180]:
    sales_train_evaluation_lag_rolling['rolling_median_'+str(i)] = sales_train_evaluation_lag_rolling.groupby('item_id')['lag_28'].transform(lambda x: x.rolling(i).median()).astype(np.float16) 

In [8]:
sales_train_evaluation_lag_rolling

,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,date_day,date_week,date_month,date_year,date_week_m,lag_28,lag_35,lag_42,lag_49,lag_56,rolling_median_7,rolling_median_14,rolling_median_30,rolling_median_60,rolling_median_180
0,1444,3,1,0,0,1,12,2011-01-29,11101,1,1,2011,30,4,4,2,0,0,0,0.46,29,4,1,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1444,3,1,0,0,2,15,2011-01-30,11101,2,1,2011,30,4,4,2,0,0,0,0.46,30,4,1,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1444,3,1,0,0,3,0,2011-01-31,11101,3,1,2011,30,4,4,2,0,0,0,0.46,31,5,1,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1444,3,1,0,0,4,0,2011-02-01,11101,4,2,2011,30,4,4,2,1,1,0,0.46,1,5,2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1444,3,1,0,0,5,0,2011-02-02,11101,5,2,2011,30,4,4,2,1,0,1,0.46,2,5,2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46851182,1432,2,0,9,2,1940,1,2016-05-21,11617,1,5,2016,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
46851183,1433,2,0,9,2,1940,1,2016-05-21,11617,1,5,2016,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
46851184,1434,2,0,9,2,1940,0,2016-05-21,11617,1,5,2016,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
46851185,1435,2,0,9,2,1940,1,2016-05-21,11617,1,5,2016,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [9]:

data_dir = ''

sales_train_evaluation_lag_rolling.to_csv(os.path.join(data_dir, 'sales_train_evaluation_lag_rolling.csv'), index=False)

sales_train_evaluation_lag_rolling_location = sagemaker_session.upload_data(os.path.join(data_dir, 'sales_train_evaluation_lag_rolling.csv'))



In [2]:
sales_train_evaluation_lag_rolling = pd.read_csv('s3://{}/{}'.format(bucket, 'data/sales_train_evaluation_lag_rolling.csv'))

KeyboardInterrupt: 

In [ ]:
sales_train_evaluation_lag_rolling

In [ ]:
sales_train_evaluation_lag_rolling = sales_train_evaluation_lag_rolling.drop(['date', 'wm_yr_wk', 'wday', 'month', 'year'])

In [ ]:
train = sales_train_evaluation_lag_rolling[sales_train_evaluation_lag_rolling['d'] < 1914]
val =  sales_train_evaluation_lag_rolling[(sales_train_evaluation_lag_rolling['d']>=1914) & (sales_train_evaluation_lag_rolling['d']<=1941)]
test = sales_train_evaluation_lag_rolling[sales_train_evaluation_lag_rolling['d'] >= 1942]


In [6]:
train.to_csv(os.path.join(data_dir, 'train_latest.csv'), index=False)

train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train_latest.csv'))


val_latest.to_csv(os.path.join(data_dir, 'val_latest.csv'), index=False)

val_latest_location = sagemaker_session.upload_data(os.path.join(data_dir, 'val_latest.csv'))


test_latest.to_csv(os.path.join(data_dir, 'test_latest.csv'), index=False)

test_latest_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test_latest.csv'))


NameError: name 'train_latest' is not defined